# Word2Vec - 영어

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

## 훈련 데이터 이해

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f8dbbad6c50>)

## 훈련 데이터 전처리

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [5]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [6]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [7]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [8]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [9]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [10]:
print(f'총 샘플의 개수 : {len(result):,d}')

총 샘플의 개수 : 273,424


In [11]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련
        size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원
        window = 컨텍스트 윈도우 크기
        min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
        workers = 학습을 위한 프로세스 수
        sg = 0은 CBOW, 1은 Skip-gram

In [12]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [13]:
model.wv.most_similar("man")

[('woman', 0.8604834675788879),
 ('guy', 0.8149069547653198),
 ('lady', 0.7484892010688782),
 ('boy', 0.7414742708206177),
 ('gentleman', 0.7393798828125),
 ('girl', 0.7311989068984985),
 ('soldier', 0.7066493630409241),
 ('kid', 0.6904914379119873),
 ('david', 0.6730118989944458),
 ('poet', 0.6604738831520081)]

In [14]:
# man - boy + girl --> woman
model.wv.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

[('woman', 0.8534905910491943),
 ('guy', 0.7464481592178345),
 ('lady', 0.7348035573959351)]

## 모델 저장하고 로드

In [15]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [16]:
!ls -l

total 97876
-rw-r--r-- 1 root root 25683092 Sep  3 04:49 eng_w2v
drwxr-xr-x 1 root root     4096 Sep  1 19:26 sample_data
-rw-r--r-- 1 root root 74533638 Sep  3 04:48 ted_en-20160408.xml


In [17]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

In [18]:
loaded_model.most_similar("man")

[('woman', 0.8604834675788879),
 ('guy', 0.8149069547653198),
 ('lady', 0.7484892010688782),
 ('boy', 0.7414742708206177),
 ('gentleman', 0.7393798828125),
 ('girl', 0.7311989068984985),
 ('soldier', 0.7066493630409241),
 ('kid', 0.6904914379119873),
 ('david', 0.6730118989944458),
 ('poet', 0.6604738831520081)]